In [1]:
import pandas as pd
import datetime
from WindPy import w
import requests
from Portfolio import portfolio
import time
import matplotlib.pyplot as plt
import easytrader
import pprint
import math

In [2]:
def rounddown(x, n):
    return int(math.floor(x / n)) * n

# 1. Connect to Wind and 华泰

In [3]:
# start WInd
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [20]:
# start 华泰
user = easytrader.use('ht_client')
user.prepare(user='666637575127', password='671109', comm_password='246810')

# 2. Get the current position

## Set up position template
## Our desired funds
### 1. 160325.SZ 华夏创业 2022/7/24
### 2. 160926.SZ 创业大成 2022/7/16
### 3. 501080.SH 科创中金 2022/7/11
### 4. 501082.SH 科创投资 2022/6/27
### 5. 506002.SH 易基科创 2022/7/28
### 6. 506003.SH 富国科创 2022/7/29
### 7. 506006.SH 添富科创 2022/7/28

In [5]:
position = {
    "160325":{
        "name":"华夏创业",
        "ticker":"160325.SZ",
        "price":w.wsq('160325.SZ', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":(datetime.datetime(2022, 7, 24, 0, 0, 0) - datetime.datetime.now()).days/365
    }, 
    "160926":{
        "name":"创业大成",
        "ticker":"160926.SZ",
        "price":w.wsq('160926.SZ', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":(datetime.datetime(2022, 7, 16, 0, 0, 0) - datetime.datetime.now()).days/365
    }, 
    "501080":{
        "name":"科创中金",
        "ticker":"501080.SH",
        "price":w.wsq('501080.SH', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":(datetime.datetime(2022, 7, 11, 0, 0, 0) - datetime.datetime.now()).days/365
    },
    "501082":{
        "name":"科创投资",
        "ticker":"501082.SH",
        "price":w.wsq('501082.SH', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":(datetime.datetime(2022, 6, 27, 0, 0, 0) - datetime.datetime.now()).days/365
    },
    "506002":{
        "name":"易基科创",
        "ticker":"506002.SH",
        "price":w.wsq('506002.SH', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":(datetime.datetime(2022, 7, 28, 0, 0, 0) - datetime.datetime.now()).days/365
    },
    "506003":{
        "name":"富国科创",
        "ticker":"506003.SH",
        "price":w.wsq('506003.SH', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":(datetime.datetime(2022, 7, 29, 0, 0, 0) - datetime.datetime.now()).days/365
    },
    "506006":{
        "name":"添富科创",
        "ticker":"506006.SH",
        "price":w.wsq('506006.SH', 'rt_last').Data[0][0],
        "shares":0,
        "expiration":(datetime.datetime(2022, 7, 28, 0, 0, 0) - datetime.datetime.now()).days/365
    },
}

In [10]:
position

{'160325': {'name': '华夏创业',
  'ticker': '160325.SZ',
  'price': 1.168,
  'shares': 0,
  'expiration': 0.9506849315068493},
 '160926': {'name': '创业大成',
  'ticker': '160926.SZ',
  'price': 1.198,
  'shares': 0,
  'expiration': 0.9287671232876712},
 '501080': {'name': '科创中金',
  'ticker': '501080.SH',
  'price': 1.747,
  'shares': 0,
  'expiration': 0.915068493150685},
 '501082': {'name': '科创投资',
  'ticker': '501082.SH',
  'price': 2.125,
  'shares': 0,
  'expiration': 0.8767123287671232},
 '506002': {'name': '易基科创',
  'ticker': '506002.SH',
  'price': 1.187,
  'shares': 0,
  'expiration': 0.9616438356164384},
 '506003': {'name': '富国科创',
  'ticker': '506003.SH',
  'price': 0.98,
  'shares': 0,
  'expiration': 0.9643835616438357},
 '506006': {'name': '添富科创',
  'ticker': '506006.SH',
  'price': 1.193,
  'shares': 0,
  'expiration': 0.9616438356164384}}

## Read 华泰 position and balance

In [16]:
ht_position = user.position

targets = [
    '506002',
    '501082',
    '506006',
    '160926',
    '506003',
    '160325',
    '501080'
]

for holding in ht_position:
    if holding['证券代码'] in targets:
        position[holding['证券代码']]['shares'] = round(holding['市值']/holding['市价'], -2)
        
fee = 0.00003

In [11]:
def strategy(interval, position, threshold):
    
    log_df = pd.DataFrame()
    
    dt = datetime.datetime.now()
    
    while dt.time() < datetime.time(11, 30):
    
        print(dt.time())
        now = datetime.datetime.now()
        
        # 1. create snapshot to document price and NAV
        snapshot = {now : {}}
        for code, info in position.items():
            
            headers = {
                'content-type': 'application/json',
                'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:22.0) Gecko/20100101 Firefox/22.0'
            }
            
            response = requests.get(f'https://fundgz.1234567.com.cn/js/{code}.js', headers=headers)
            NAV = float(eval(response.text.replace('jsonpgz(', '').replace(');', ''))['gsz'])
            
            Price = w.wsq(info['ticker'], 'rt_last').Data[0][0]
            info['price'] = Price
            AnnualizedRateOfReturn = round(((NAV/Price)**(1/info['expiration'])-1)*100, 2)
            #NAV, Price, AnnualizedRateOfReturn
            snapshot[now][code + ' NAV'] = NAV
            snapshot[now][code + ' Price'] = Price
            snapshot[now][code + ' AnnualizedRateOfReturn (%)'] = AnnualizedRateOfReturn
        
        in_ranks = {k:v for k,v in snapshot[now].items() if "AnnualizedRateOfReturn (%)" in k and position[k.split(' ')[0]]['shares'] != 0}
        out_ranks = {k:v for k,v in snapshot[now].items() if "AnnualizedRateOfReturn (%)" in k and position[k.split(' ')[0]]['shares'] == 0}
        in_ranks = {k.split(' ')[0]:snapshot[now][k] for k in sorted(in_ranks, key=in_ranks.get, reverse=True)}        
        out_ranks = {k.split(' ')[0]:snapshot[now][k] for k in sorted(out_ranks, key=out_ranks.get, reverse=True)}
        print("Holdings: ")
        print(in_ranks)
        print("Not in holdings: ")
        print(out_ranks)
            
        # 2. Trade
        if list(out_ranks.items())[0][1] - list(in_ranks.items())[-1][1] >= threshold:
            print(f"Sell {list(in_ranks.items())[-1][0]}")
            sale = list(in_ranks.items())[-1][0]
            sell_price = position[sale]['price'] - 0.001
            # sell_result = user.sell(sale, price=sell_price, amount=position[sale]['shares'])
            print("Sell", sell_result)
            
#             while True:
#                 for fund in user.position:
#                     if fund['证券代码'] == list(in_ranks.items())[-1][0]:
#                         check_sale = fund
                
#                 if check_sale['股票余额'] == 0:
#                     break
#                 else:
#                     user.cancel_entrust(sell_result["entrust_no"])
#                     sell_price = sell_price - 0.001
#                     sell_result = user.sell(sale, price=sell_price, amount=check_sale['股票余额'])

#                 time.sleep(60*3)
                            
#             purchase_amount = rounddown((user.balance['可用金额'] * fee)/(position[purchase]['price']+0.001), 100)
#             buy_result = user.buy(purchase, price=position[purchase]['price']+0.001, amount=purchase_amount)
#             print(buy_result)
            
            print(f"Buy {list(out_ranks.items())[0][0]}")
            
        ht_position = user.position

        targets = [
            '506002',
            '501082',
            '506006',
            '160926',
            '506003',
            '160325',
            '501080'
        ]
 
        for holding in ht_position:
            if holding['证券代码'] in targets:
                position[holding['证券代码']]['shares'] = round(holding['市值']/holding['市价'], -2)
                
        position = position
        portfolio.cash = user.balance['可用金额']
        
        dt = datetime.datetime.now()
        
        log_df = pd.concat([log_df, pd.DataFrame(snapshot).T], axis=0)
    
        time.sleep(interval)
        
    return log_df

In [22]:
threshold = 1.5
interval = 60

In [27]:
dt = datetime.datetime.now()

while dt.time() < datetime.time(11, 30):

    print(dt.time())
    now = datetime.datetime.now()

    # 1. create snapshot to document price and NAV
    snapshot = {now : {}}
    for code, info in position.items():

        headers = {
            'content-type': 'application/json',
            'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:22.0) Gecko/20100101 Firefox/22.0'
        }

        response = requests.get(f'https://fundgz.1234567.com.cn/js/{code}.js', headers=headers)
        NAV = float(eval(response.text.replace('jsonpgz(', '').replace(');', ''))['gsz'])

        Price = w.wsq(info['ticker'], 'rt_last').Data[0][0]
        info['price'] = Price
        AnnualizedRateOfReturn = round(((NAV/Price)**(1/info['expiration'])-1)*100, 2)
        #NAV, Price, AnnualizedRateOfReturn
        snapshot[now][code + ' NAV'] = NAV
        snapshot[now][code + ' Price'] = Price
        snapshot[now][code + ' AnnualizedRateOfReturn (%)'] = AnnualizedRateOfReturn

    in_ranks = {k:v for k,v in snapshot[now].items() if "AnnualizedRateOfReturn (%)" in k and position[k.split(' ')[0]]['shares'] != 0}
    out_ranks = {k:v for k,v in snapshot[now].items() if "AnnualizedRateOfReturn (%)" in k and position[k.split(' ')[0]]['shares'] == 0}
    in_ranks = {k.split(' ')[0]:snapshot[now][k] for k in sorted(in_ranks, key=in_ranks.get, reverse=True)}        
    out_ranks = {k.split(' ')[0]:snapshot[now][k] for k in sorted(out_ranks, key=out_ranks.get, reverse=True)}
    print("Holdings: ")
    print(in_ranks)
    print("Not in holdings: ")
    print(out_ranks)

    # 2. Trade
    if list(out_ranks.items())[0][1] - list(in_ranks.items())[-1][1] >= threshold:
        sale = list(in_ranks.items())[-1][0]
        purchase = list(out_ranks.items())[0][0]
        print(f"Sell {sale} Buy {purchase}")
        
        sell_price = position[sale]['price']
        sell_result = user.sell(sale, price=sell_price, amount=check_sale['股票余额'])
        print("Sell: ", sell_result)
        
        while True:
            for fund in user.position:
                if fund['证券代码'] == sale:
                    check_sale = fund

            if check_sale['股票余额'] == 0:
                break
            else:
                user.cancel_entrust(sell_result["entrust_no"])
                sell_price = sell_price - 0.001
                sell_result = user.sell(sale, price=sell_price, amount=check_sale['股票余额'])
                print("Sell: ", sell_result)

            time.sleep(60*3)

        buy_price = position[purchase]['price']
        purchase_amount = rounddown((user.balance['可用金额'] * fee)/buy_price, 100)
        buy_result = user.buy(purchase, price=buy_price, amount=purchase_amount)
        print("Buy: ", buy_result) 
        
        while True:
            for fund in user.position:
                if fund['证券代码'] == purchase:
                    check_purchase = fund

            purchase_amount = rounddown((user.balance['可用金额'] * fee)/buy_price, 100)
    
            if purchase_amount == 0:
                break
            else:
                user.cancel_entrust(buy_result["entrust_no"])
                buy_price = buy_price + 0.001
                buy_result = user.buy(purchase, price=buy_price, amount=purchase_amount)
                print("Buy: ", buy_result)

            time.sleep(60*3)

    ht_position = user.position

    targets = [
        '506002',
        '501082',
        '506006',
        '160926',
        '506003',
        '160325',
        '501080'
    ]

    for holding in ht_position:
        if holding['证券代码'] in targets:
            position[holding['证券代码']]['shares'] = round(holding['市值']/holding['市价'], -2)

    dt = datetime.datetime.now()

    log_df = pd.concat([log_df, pd.DataFrame(snapshot).T], axis=0)

    time.sleep(interval)

11:02:59.853800
Holdings: 
{'160926': 14.15, '160325': 14.03, '506003': 13.14, '506002': 12.75}
Not in holdings: 
{'506006': 13.85, '501082': 12.89, '501080': 11.29}
11:03:06.169800
Holdings: 
{'160926': 14.02, '160325': 13.89, '506003': 12.91, '506002': 12.79}
Not in holdings: 
{'506006': 13.85, '501082': 13.14, '501080': 11.29}
11:04:12.551800
Holdings: 
{'160926': 13.93, '160325': 13.72, '506003': 12.98, '506002': 12.76}
Not in holdings: 
{'506006': 13.47, '501082': 12.9, '501080': 11.21}
11:05:19.694800
Holdings: 
{'160926': 13.99, '160325': 13.82, '506003': 12.81, '506002': 12.51}
Not in holdings: 
{'506006': 13.4, '501082': 12.6, '501080': 11.05}
11:06:27.620800
Holdings: 
{'160926': 13.99, '160325': 13.67, '506003': 12.81, '506002': 12.63}
Not in holdings: 
{'506006': 13.37, '501082': 12.48, '501080': 10.92}
11:07:36.452800
Holdings: 
{'160926': 13.87, '160325': 13.67, '506003': 12.78, '506002': 12.66}
Not in holdings: 
{'506006': 13.31, '501082': 12.45, '501080': 10.92}
11:08:4

IndexError: list index out of range

In [28]:
log_df

,160325 AnnualizedRateOfReturn (%),160325 NAV,160325 Price,160926 AnnualizedRateOfReturn (%),160926 NAV,160926 Price,501080 AnnualizedRateOfReturn (%),501080 NAV,501080 Price,501082 AnnualizedRateOfReturn (%),...,501082 Price,506002 AnnualizedRateOfReturn (%),506002 NAV,506002 Price,506003 AnnualizedRateOfReturn (%),506003 NAV,506003 Price,506006 AnnualizedRateOfReturn (%),506006 NAV,506006 Price
2021-08-10 10:49:38.010800,13.92,1.3277,1.173,14.22,1.3600,1.202,11.49,1.9299,1.747,12.94,...,2.130,12.78,1.3359,1.190,13.43,1.1066,0.980,14.00,1.3543,1.194
2021-08-10 10:50:44.207800,13.88,1.3261,1.172,14.02,1.3578,1.202,11.35,1.9276,1.747,12.94,...,2.130,12.70,1.3327,1.188,12.94,1.1043,0.982,13.74,1.3514,1.194
2021-08-10 10:51:50.456800,13.84,1.3257,1.172,14.12,1.3578,1.201,11.39,1.9283,1.747,12.70,...,2.130,12.70,1.3327,1.188,12.94,1.1043,0.982,13.74,1.3514,1.194
2021-08-10 10:52:56.664800,13.84,1.3257,1.172,14.02,1.3578,1.202,11.37,1.9279,1.747,12.77,...,2.130,12.78,1.3337,1.188,12.94,1.1043,0.982,13.89,1.3519,1.193
2021-08-10 10:54:02.770800,13.84,1.3257,1.172,14.21,1.3587,1.201,11.42,1.9287,1.747,12.76,...,2.130,12.77,1.3336,1.188,12.94,1.1043,0.982,13.78,1.3518,1.194
2021-08-10 10:55:10.107800,13.90,1.3264,1.172,14.10,1.3587,1.202,11.48,1.9296,1.747,12.76,...,2.130,12.77,1.3335,1.188,12.92,1.1041,0.982,13.87,1.3528,1.194
2021-08-10 10:59:08.215800,14.03,1.3255,1.170,14.23,1.3578,1.200,11.50,1.9300,1.747,12.68,...,2.128,12.57,1.3313,1.188,13.05,1.1042,0.981,13.89,1.3520,1.193
2021-08-10 11:00:14.422800,14.10,1.3263,1.170,14.23,1.3578,1.200,11.61,1.9295,1.745,12.68,...,2.128,12.57,1.3313,1.188,13.05,1.1042,0.981,13.90,1.3521,1.193
2021-08-10 11:02:59.855800,14.03,1.3255,1.170,14.15,1.3569,1.200,11.29,1.9289,1.749,12.89,...,2.124,12.75,1.3322,1.187,13.14,1.1039,0.980,13.85,1.3515,1.193
2021-08-10 11:04:06.171800,13.89,1.3240,1.170,14.02,1.3555,1.200,11.29,1.9289,1.749,13.14,...,2.120,12.79,1.3315,1.186,12.91,1.1017,0.980,13.85,1.3515,1.193
